In [ ]:
pip install transformers sentencepiece

In [ ]:
pip install transformers

In [1]:
import torch
import os
from glob import glob
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
import math
from tqdm import tqdm
from ast import literal_eval
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import time
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import T5ForConditionalGeneration, T5Tokenizer
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR, SequentialLR
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.modeling_outputs import BaseModelOutput
import os
from typing import List, Tuple
from glob import glob
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



In [2]:
# ---------------------
# Load T5-Base Model
# ---------------------
summarizer_model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
# Load nid2body from pickle
with open("nid2body.pkl", "rb") as f:
    nid2body = pickle.load(f)

# Debug print
print(f"✅ Loaded nid2body with {len(nid2body)} items")
sample_nid = list(nid2body.keys())[0]
print(f"🧾 Sample NID: {sample_nid}\n📝 Headline: {nid2body[sample_nid][:300]}")

✅ Loaded nid2body with 113762 items
🧾 Sample NID: N10000
📝 Headline: Only FIVE internationals allowed, count em, FIVE! So first off we should say, per our usual Atlanta United lineup predictions, this will be wrong. Why will it be wrong? Well, aside from the obvious, we still don't have a ton of data points from Frank de Boer in how he prefers to rotate his team for 


In [4]:
# Load sid2sum from pickle
with open("sid2sum.pkl", "rb") as f:
    sid2sum = pickle.load(f)

# Debug print
print(f"✅ Loaded sid2sum with {len(sid2sum)} items")
sample_sid = list(sid2sum.keys())[0]
print(f"🧾 Sample SID: {sample_sid}\n📝 Summary: {sid2sum[sample_sid][:300]}")

✅ Loaded sid2sum with 135001 items
🧾 Sample SID: S-1
📝 Summary: The officer reportedly also pointed his gun at Harper and her children.


In [5]:
# === Device and Precision Setup ===
torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 768

# === Utility Functions ===
def get_embedding(key, table, dim):
    if key not in table:
        table[key] = torch.nn.Parameter(torch.randn(dim, dtype=torch.float32, device=device) * 0.01, requires_grad=True)
    return table[key]


# === Load Embeddings ===
with open("summary_T5.pkl", "rb") as f:
    summary_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}
with open("newsbody_T5.pkl", "rb") as f:
    newsbody_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}
with open("headline_T5.pkl", "rb") as f:
    headline_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}

embed_tables = {
    'summary': summary_embed,
    'newsbody': newsbody_embed,
    'headline': headline_embed
}

# === Load Dataset ===
lookup_df = pd.read_csv("w2p_engage_list.csv").set_index('EdgeID')
train_df = pd.read_csv("train_df_gold_only.csv")

In [6]:
tqdm.pandas()  # enable progress_apply

# Convert all Bhist strings to actual lists with progress bar
train_df['EHist'] = train_df['EHist'].progress_apply(literal_eval)

# Check a few entries
print(train_df['EHist'].head())

# Compute max length
max_len = max(len(h) for h in train_df["EHist"])
max_len_plus_one = max_len + 1
print("Max length + 1:", max_len_plus_one)

100%|██████████| 38417/38417 [00:03<00:00, 11439.69it/s]

0    [E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11,...
1    [E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11,...
2                                               [E151]
3    [E151, E152, E153, E154, E155, E156, E157, E15...
4    [E151, E152, E153, E154, E155, E156, E157, E15...
Name: EHist, dtype: object
Max length + 1: 384


In [ ]:
PAD_TOKEN = "<PAD>"   # or 0 if you want integer padding later
max_len = max(len(h) for h in train_df["EHist"])
print("Max length:", max_len)

# Compute original lengths before padding
train_df["EHist_len"] = train_df["EHist"].apply(len)

# Then pad as before
PAD_TOKEN = "<PAD>"
train_df["EHist_padded"] = train_df["EHist"].progress_apply(
    lambda h: h + [PAD_TOKEN] * (max_len - len(h))
)


# Check shapes
print(train_df["EHist_padded"].iloc[0])
print(len(train_df["EHist_padded"].iloc[0]))  # should equal max_len

Max length: 383


100%|██████████| 38417/38417 [00:00<00:00, 158023.31it/s]

['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20', 'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29', 'E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46', 'E47', 'E48', 'E49', 'E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64', 'E65', 'E66', 'E67', 'E68', 'E69', 'E70', 'E71', 'E72', 'E73', 'E74', 'E75', 'E76', 'E77', 'E78', 'E79', 'E80', 'E81', 'E82', 'E83', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PA

In [8]:
train_df

,Unnamed: 0,UserID,EHist,EPos,EHist_len,EHist_padded,Tail
0,2,U10000_1,"[E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11,...",E84,83,"[E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11,...",S-1
1,3,U10000_2,"[E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11,...",E133,132,"[E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11,...",S-2
2,11,U100006_1,[E151],E152,1,"[E151, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD...",S-3
3,12,U100006_2,"[E151, E152, E153, E154, E155, E156, E157, E15...",E168,17,"[E151, E152, E153, E154, E155, E156, E157, E15...",S-4
4,13,U100006_3,"[E151, E152, E153, E154, E155, E156, E157, E15...",E230,79,"[E151, E152, E153, E154, E155, E156, E157, E15...",S-5
...,...,...,...,...,...,...,...
38412,157388,U20775_1,"[E1512010, E1512011, E1512012, E1512013, E1512...",E1512050,40,"[E1512010, E1512011, E1512012, E1512013, E1512...",S-38704
38413,157389,U20775_2,"[E1512010, E1512011, E1512012, E1512013, E1512...",E1512064,54,"[E1512010, E1512011, E1512012, E1512013, E1512...",S-38705
38414,157390,U20775_3,"[E1512010, E1512011, E1512012, E1512013, E1512...",E1512112,102,"[E1512010, E1512011, E1512012, E1512013, E1512...",S-38706
38415,157391,U20775_4,"[E1512010, E1512011, E1512012, E1512013, E1512...",E1512115,105,"[E1512010, E1512011, E1512012, E1512013, E1512...",S-38707


In [9]:
# ======================================================
# BehaviorEncoder (uses padded Bhist + Bhist_len)
# ======================================================
class BehaviorEncoder(nn.Module):
    def __init__(self, hidden_dim, device, max_len, debug=False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.debug = debug
        self.max_len = max_len

        # === Base action vectors ===
        self.e_clk = nn.Parameter(torch.tensor([1., 0., 0., 0.], device=device))
        self.e_skp = nn.Parameter(torch.tensor([0., 1., 0., 0.], device=device))
        self.e_gensumm = nn.Parameter(torch.tensor([0., 0., 1., 0.], device=device))
        self.e_sumgen = nn.Parameter(torch.tensor([0., 0., 0., 1.], device=device))

        # Action-specific transforms
        self.W_clk = nn.Linear(4, hidden_dim, bias=False)
        self.W_skp = nn.Linear(4, hidden_dim, bias=False)
        self.W_gensumm = nn.Linear(4, hidden_dim, bias=False)
        self.W_sumgen = nn.Linear(4, hidden_dim, bias=False)

        # State transforms
        self.W_pull = nn.Linear(1, hidden_dim, bias=False)
        self.W_s = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_d = nn.Linear(hidden_dim, hidden_dim, bias=False)

        # Fusion
        self.Wh = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.Wc = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.Wz = nn.Linear(hidden_dim, 3, bias=False)
        self.b_z = nn.Parameter(torch.zeros(3, device=device))
        self.W_emb = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.b_emb = nn.Parameter(torch.zeros(hidden_dim, device=device))  # kept as in your design

        # Rotation/translation
        self.W_angle = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_theta = nn.Linear(hidden_dim, 1, bias=False)
        self.W_h = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_m = nn.Linear(hidden_dim, 1, bias=False)

        # Scalars
        self.alpha = nn.Parameter(torch.tensor(0.5, device=device))
        self.beta = nn.Parameter(torch.tensor(0.5, device=device))

        # Classifier head over timesteps (0...max_len+1)
        self.classifier = nn.Linear(hidden_dim, max_len + 2)

        # Next-step prediction head
        self.W_next = nn.Linear(hidden_dim, hidden_dim, bias=False)

    def softmin_pool(self, a, b):
        return -self.alpha * torch.log(torch.exp(a / self.alpha) +
                                       torch.exp(b / self.alpha) + 1e-9)

    def forward(self, Bhist, Bhist_len, Bpos, lookup_df, embed_tables):
        """
        Inputs:
          - Bhist: padded list/sequence of EIDs (length == self.max_len)
          - Bhist_len: original unpadded length (int)
          - Bpos: target EID (scalar id)
          - lookup_df: DataFrame indexed by EID with columns 'Tail','Head','Relation'
          - embed_tables: dict with keys 'newsbody','summary','headline' mapping item-id -> tensor(hidden_dim,)
        Outputs:
          - eprime_last: final contextualized embedding (for last real token)
          - eprime_next: predicted next-step embedding (W_next(eprime_last))
          - logits_pos: classifier logits for the next-step (vector of size max_len+2)
          - ctx_enc_loss: accumulated stepwise classification loss (sum over real steps)
          - total_loss: weighted total loss (0.2 * step-loss + 0.8 * next-pos CE) as you had
        Notes:
          - This function explicitly iterates only over the first Bhist_len entries (ignoring PADs).
        """
        # ensure Bhist is indexable (list/1-D tensor). If tensor, bring to cpu/int list for lookup indexing
        if isinstance(Bhist, torch.Tensor):
            Bhist_list = Bhist.detach().cpu().tolist()
        else:
            Bhist_list = list(Bhist)

        # initialize losses on device
        ctx_enc_loss = torch.tensor(0., dtype=torch.float32, device=self.device)

        # PASS 1: build per-step raw embeddings E_seq for real tokens only
        E_seq = []
        h_clk = torch.zeros(self.hidden_dim, device=self.device)
        h_skp = torch.zeros(self.hidden_dim, device=self.device)
        h = torch.zeros(self.hidden_dim, device=self.device)

        for t in range(int(Bhist_len)):  # iterate only real tokens
            b_id = Bhist_list[t]
            # skip if missing in lookup
            if b_id not in lookup_df.index:
                continue

            row = lookup_df.loc[b_id]
            tail_id, rel = row['Tail'], row['Relation']

            # fetch embeddings (fallback zeros)
            d_i = embed_tables['newsbody'].get(tail_id, torch.zeros(self.hidden_dim, device=self.device))
            s_i = embed_tables['summary'].get(tail_id, torch.zeros(self.hidden_dim, device=self.device))
            d_i_title = embed_tables['headline'].get(tail_id, torch.zeros(self.hidden_dim, device=self.device))

            # initialize at t==0
            if t == 0:
                head_emb = embed_tables['headline'].get(tail_id, torch.zeros(self.hidden_dim, device=self.device))
                h_clk, h_skp = head_emb.clone(), head_emb.clone()
                # keep original math: W_s(head_emb) * h_clk + (1-W_s(head_emb)) * h_skp
                h = torch.tanh(self.W_s(head_emb) * h_clk + (1 - (self.W_s(head_emb))) * h_skp)

            # relation-specific context
            if rel == "click":
                c_i = torch.tanh(self.W_clk.weight @ self.e_clk * h) * d_i
            elif rel == "skip":
                d_ip1 = torch.zeros_like(d_i)
                if (t + 1) < Bhist_len and Bhist_list[t+1] in lookup_df.index:
                    next_row = lookup_df.loc[Bhist_list[t+1]]
                    d_ip1 = embed_tables['newsbody'].get(next_row['Head'], torch.zeros_like(d_i))
                pull_term = self.W_pull(torch.tensor([[torch.dot(h_clk, d_ip1) + torch.dot(h_skp, d_i)]],
                                                     device=self.device)).squeeze(0)
                c_i = -torch.tanh(self.W_skp.weight @ self.e_skp + d_i + pull_term * h) * d_i
            elif rel == "gen_summ":
                c_i = torch.tanh(self.W_gensumm.weight @ self.e_gensumm * h) * d_i_title
            elif rel == "summ_gen":
                gate_summgen = torch.tanh(self.W_s(self.W_sumgen.weight @ self.e_sumgen))
                c_i = self.softmin_pool(gate_summgen * s_i, (1 - gate_summgen) * d_i)
                h_clk = h_clk + self.W_d((torch.ones_like(d_i_title) - d_i_title) * s_i)
            else:
                c_i = d_i

            # hidden update
            z_i = torch.tanh(self.Wh(h) + self.Wc(c_i))
            p_i = torch.softmax(self.Wz(z_i) + self.b_z, dim=-1)
            m_i = p_i[0] * 0.1 + p_i[1] * 0.5 + p_i[2] * 0.9
            if rel == "click":
                h_clk = h_clk + m_i * c_i
            elif rel == "skip":
                h_skp = h_skp * (1 - m_i) + c_i
            h = (self.beta * h_clk + (1 - self.beta) * h_skp)

            e_i = torch.tanh(self.W_emb(c_i))
            E_seq.append(e_i)

        # if E_seq is empty (no valid history), return zeros to avoid crash
        if len(E_seq) == 0:
            zero_e = torch.zeros(self.hidden_dim, device=self.device)
            eprime_last = zero_e
            eprime_next = self.W_next(eprime_last)
            logits_pos = self.classifier(eprime_next.unsqueeze(0))
            ctx_enc_loss = torch.tensor(0., device=self.device)
            total_loss = torch.tensor(0., device=self.device)
            return eprime_last, eprime_next, logits_pos, ctx_enc_loss, total_loss

        # PASS 2: contextualize E_seq -> Eprime_seq (only for real tokens)
        Eprime_seq = []
        eps = 1e-6
        for i, e_i in enumerate(E_seq):
            if i == 0:
                e_prime = e_i
            else:
                e_prev, e_prime_prev = E_seq[i-1], Eprime_seq[-1]
                theta_i = math.pi * torch.tanh(self.W_theta(torch.sigmoid(self.W_angle(e_prime_prev))))
                m_i = F.softplus(self.W_m(self.W_h(e_prime_prev)))

                # compute orthonormal direction and rotation (as in your original)
                v_i = (e_i - e_prime_prev) / (e_i - e_prime_prev).norm(p=2).clamp(min=eps)
                u_prev = e_prev / e_prime_prev.norm(p=2).clamp(min=eps)
                o_i = (v_i - torch.dot(v_i, u_prev) * u_prev)
                o_i = o_i / o_i.norm(p=2).clamp(min=eps)

                e_prime = e_prime_prev + m_i * (torch.cos(theta_i) * u_prev + torch.sin(theta_i) * o_i).squeeze()
                e_prime = torch.tanh(e_prime) + e_i

            Eprime_seq.append(e_prime)

            # per-step classification loss only for real steps
            target_step = torch.tensor([i], device=self.device)
            logits_step = self.classifier(e_prime.unsqueeze(0))
            ctx_enc_loss = ctx_enc_loss + F.cross_entropy(logits_step, target_step)

        # Final prediction on Bpos (next-step)
        eprime_last = Eprime_seq[-1]
        eprime_next = self.W_next(eprime_last)
        logits_pos = self.classifier(eprime_next.unsqueeze(0))

        # target for next-position is Bhist_len (0-based indexing consistent with your design)
        target_pos = torch.tensor([int(Bhist_len)], device=self.device)
        total_loss = 0.2 * (ctx_enc_loss / max(1, float(Bhist_len))) + 0.8 * F.cross_entropy(logits_pos, target_pos)

        return eprime_last, eprime_next, logits_pos, ctx_enc_loss, total_loss

In [10]:
# ======================================================
# BehaviorInverseDecoderPredict
# ======================================================
class BehaviorInverseDecoderPredict(nn.Module):
    """
    Inverse mapping that takes a single predicted e' (embedding for Bpos)
    and the document embedding for Bpos, and returns:
      - c'_pos (approx pseudo-content)
      - s_hat_pos (approx summary)
    """
    def __init__(self, hidden_dim, device, debug=False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.debug = debug

        # Learnable pseudo-inverse / disentanglers
        self.W_emb_pinv = nn.Linear(hidden_dim, hidden_dim, bias=False)   # approx W_emb^+
        self.W1_pinv = nn.Linear(hidden_dim, hidden_dim, bias=False)      # remove head contribution
        self.W2_pinv = nn.Linear(hidden_dim, hidden_dim, bias=False)      # map residual -> summary

    def _show(self, name, tensor, maxlen=6):
        if not self.debug:
            return
        if isinstance(tensor, torch.Tensor):
            if tensor.ndim == 0:
                print(f"{name}: {tensor.item():.6f}")
            else:
                flat = tensor.detach().cpu().numpy().flatten()
                vals = ", ".join(f"{x:.6f}" for x in flat[:maxlen])
                if len(flat) > maxlen: vals += ", ..."
                print(f"{name} (shape={tuple(tensor.shape)}): [{vals}]")
        else:
            print(f"{name}: {tensor}")

    @staticmethod
    def atanh_safe(x, eps=1e-6):
        x = x.clamp(-1+eps, 1-eps)
        return 0.5 * torch.log((1+x) / (1-x))

    def forward(self, eprime_pos, b_emb, h_pos):
        """
        eprime_pos: tensor (hidden_dim,) -- predicted e' embedding for Bpos (assumed already on device)
        b_emb: encoder bias b_emb (tensor (hidden_dim,))
        h_pos: head/headline embedding for Bpos (tensor (hidden_dim,))
        Returns: c_prime_pos, s_hat_pos  (both normalized)
        """
        # 1) invert embedding nonlinearity: atanh(e') - b_emb
        x = self.atanh_safe(eprime_pos) - b_emb  # (hidden_dim,)
        c_prime_pos = self.W_emb_pinv(x)         # approx c'_pos
        c_prime_pos = F.normalize(c_prime_pos, p=2, dim=0)

        # 2) subtract head contribution and map to summary
        residual = c_prime_pos - self.W1_pinv(h_pos)
        s_hat_pos = self.W2_pinv(residual)
        s_hat_pos = F.normalize(s_hat_pos, p=2, dim=0)

        return c_prime_pos, s_hat_pos


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.modeling_outputs import BaseModelOutput

class PersonalizedT5Summarizer(nn.Module):
    """
    TRAIN (TF) returns:
      { 'skip', 'total_loss', 'enc_loss', 'gen_loss', 'lm_logits',
        'doc_text', 'gold_summary', 'pred_summary_tf' }
    EVAL (AR) returns:
      { 'pred_summary_ar', 'enc_loss', 'doc_text', 'gold_summary' }

    Notes:
    - Supports two input modes:
      (A) Pre-tokenized (recommended for speed):
          pass enc_input_ids, dec_input_ids, dec_labels (all [B,T] or [1,T])
      (B) Legacy (fallback): pass Bpos + tokenizer + nid2body/sid2sum, and it will tokenize inside.
    - Greedy preview during training is OFF by default; enable every N steps with preview_stride>0.
    """

    def __init__(self, hidden_dim, t5_model, behavior_encoder, inverse_decoder, device,
                 learnable_ctx=True, unfreeze_last_decoder_blocks=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.t5 = t5_model                    # don't force eval() here
        self.behavior_encoder = behavior_encoder
        self.inverse_decoder = inverse_decoder
        self.device = device

        self.t5_hidden_dim = self.t5.config.d_model

        # light projection layers for context injection
        self.e_proj = nn.Linear(hidden_dim, self.t5_hidden_dim)
        self.q_proj = nn.Linear(self.t5_hidden_dim, self.t5_hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, self.t5_hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, self.t5_hidden_dim)

        if learnable_ctx:
            self.ctx_scale_raw = nn.Parameter(torch.tensor(0.1))
        else:
            self.register_buffer("ctx_scale_raw", torch.tensor(0.1), persistent=False)

        # === Freeze all, then selectively unfreeze ===
        for p in self.t5.parameters():
            p.requires_grad = False

        # Unfreeze last N decoder blocks
        if hasattr(self.t5, "decoder") and hasattr(self.t5.decoder, "block"):
            if unfreeze_last_decoder_blocks is not None and unfreeze_last_decoder_blocks > 0:
                for layer in self.t5.decoder.block[-unfreeze_last_decoder_blocks:]:
                    for p in layer.parameters():
                        p.requires_grad = True

        # Unfreeze final decoder layer norm if present
        if hasattr(self.t5, "decoder") and hasattr(self.t5.decoder, "final_layer_norm"):
            self.t5.decoder.final_layer_norm.weight.requires_grad = True

        # T5 weight tying: shared <-> lm_head must both be trainable
        if hasattr(self.t5, "shared"):
            self.t5.shared.weight.requires_grad = True
        if hasattr(self.t5, "lm_head"):
            self.t5.lm_head.weight.requires_grad = True

    # ---------- helpers ----------
    def _behavior_encode(self, Bhist, Bhist_len, Bpos, lookup_df, embed_tables):
        e_last, e_next, _, _, enc_loss = self.behavior_encoder(
            Bhist, Bhist_len, Bpos, lookup_df, embed_tables
        )
        head_id = lookup_df.loc[Bpos]['Head']
        head_emb = embed_tables['headline'].get(
            head_id, torch.zeros(self.hidden_dim, device=self.device)
        )
        b_emb = getattr(
            self.behavior_encoder, "b_emb",
            torch.zeros(self.hidden_dim, device=self.device)
        )
        _, s_hat = self.inverse_decoder(e_next, b_emb, head_emb)
        return e_last, s_hat, enc_loss

    def _encode_query(self, Bpos, lookup_df, nid2body, tokenizer, max_len):
        head_id = lookup_df.loc[Bpos]['Head']
        tail_id = lookup_df.loc[Bpos]['Tail']
        doc_text = nid2body.get(head_id, "")
        query_text = "Generate headline: " + doc_text
        # legacy tokenization (slow) — use only if pre-tokenized inputs are not given
        query_tokens = tokenizer(
            query_text,
            return_tensors="pt",
            truncation=True,
            max_length=max_len,
            padding="max_length"
        ).input_ids.to(self.device)
        enc_outs = self.t5.encoder(input_ids=query_tokens)
        base_enc = enc_outs.last_hidden_state  # [1, L, d]
        return base_enc, doc_text, tail_id

    def _encode_query_with_ids(self, enc_input_ids):
        # Fast path: pre-tokenized inputs [B,L] (or [1,L])
        enc_outs = self.t5.encoder(input_ids=enc_input_ids)
        return enc_outs.last_hidden_state  # [B, L, d]

    def _inject_context(self, base_enc, e_last, s_hat):
        # gate by user embedding
        if e_last.dim() == 1:
            e_last_batch = e_last.unsqueeze(0)
        else:
            e_last_batch = e_last
        gate = torch.sigmoid(self.e_proj(e_last_batch) / 10.0)  # [B, d]
        gate = gate.unsqueeze(1).expand(-1, base_enc.size(1), -1)  # [B, L, d]
        gated_enc = base_enc * gate

        # cross-attn from single s_hat vector
        Q = self.q_proj(gated_enc)  # [B, L, d]
        if s_hat.dim() == 1:
            s_hat_batch = s_hat.unsqueeze(0)  # [B=1, h]
        else:
            s_hat_batch = s_hat                                    # [B, h]
        K = self.k_proj(s_hat_batch).unsqueeze(1)  # [B, 1, d]
        V = self.v_proj(s_hat_batch).unsqueeze(1)  # [B, 1, d]

        attn_scores = torch.matmul(Q / (self.t5_hidden_dim ** 0.5), K.transpose(-1, -2))  # [B, L, 1]
        attn_scores = torch.clamp(attn_scores, -10, 10)
        attn_wts = F.softmax(attn_scores, dim=1)
        ctx = torch.matmul(attn_wts, V)  # [B, L, d]

        # cap the scale to avoid exploding context injection
        ctx_scale = torch.clamp(F.softplus(self.ctx_scale_raw), max=10.0)
        personalized_enc = gated_enc + ctx_scale * ctx

        # safety: if NaNs arise, fall back to gated_enc
        if not torch.isfinite(personalized_enc).all():
            personalized_enc = gated_enc
        return personalized_enc

    # ---------- public API ----------
    def forward(
        self,
        Bhist,
        Bhist_len,
        Bpos,
        lookup_df,
        embed_tables,
        sid2sum,
        nid2body,
        tokenizer,
        max_len=50,
        mode=None,
        gen_kwargs=None,
        # NEW: fast path (pre-tokenized inputs)
        enc_input_ids=None,   # [B,L] LongTensor on device
        dec_input_ids=None,   # [B,T] LongTensor on device (gold[:, :-1])
        dec_labels=None,      # [B,T] LongTensor on device (gold[:, 1:])
        # NEW: gated preview to avoid step-time explosion
        preview_stride: int = 0,
        global_step: int | None = None,
    ):
        """
        If enc_input_ids/dec_input_ids/dec_labels are provided, the model will use them directly.
        Otherwise it will tokenize internally (legacy behavior, slower).
        """

        # Make sure decoder is in train() if we're training
        if self.training and hasattr(self.t5, "decoder"):
            self.t5.decoder.train()

        # Behavior/user context
        e_last, s_hat, enc_loss = self._behavior_encode(
            Bhist, Bhist_len, Bpos, lookup_df, embed_tables
        )

        # Encoder path
        doc_text = ""
        gold_summary_text = ""
        if enc_input_ids is not None:
            # batched fast path
            base_enc = self._encode_query_with_ids(enc_input_ids)         # [B,L,d]
            # doc_text and gold_summary_text are not used in this path for loss; keep blank
            # gold text still used in logging if you want — pass it separately via caller if needed
            tail_id = lookup_df.loc[Bpos]['Tail']
            gold_summary_text = sid2sum.get(tail_id, "")
        else:
            # legacy path (tokenize per-sample)
            base_enc, doc_text, tail_id = self._encode_query(
                Bpos, lookup_df, nid2body, tokenizer, max_len
            )
            gold_summary_text = sid2sum.get(tail_id, "")

        # Personalize
        # If base_enc is [L,d] (legacy), add batch dim. If [B,L,d], keep as-is.
        if base_enc.dim() == 2:
            base_enc = base_enc.unsqueeze(0)
        personalized_enc = self._inject_context(base_enc, e_last, s_hat)

        # Mode
        if mode is None:
            mode = "train" if self.training else "ar"
        mode = mode.lower()

        # -------------------------------
        # Teacher Forcing (training)
        # -------------------------------
        if mode in ("train", "tf", "teacher", "teacher_forcing"):
            # Guards only in legacy tokenization path
            if enc_input_ids is None:
                if gold_summary_text is None or gold_summary_text.strip() == "":
                    return {
                        "skip": True,
                        "reason": "empty_gold",
                        "total_loss": torch.tensor(0.0, device=self.device),
                        "enc_loss": enc_loss,
                        "gen_loss": torch.tensor(0.0, device=self.device),
                        "lm_logits": None,
                        "doc_text": doc_text,
                        "gold_summary": "",
                        "pred_summary_tf": "",
                    }

                gold_tokens = tokenizer(
                    gold_summary_text,
                    return_tensors="pt",
                    truncation=True,
                    max_length=max_len,
                    padding="max_length"
                ).input_ids.to(self.device)  # [1, L]

                dec_input_ids = gold_tokens[:, :-1]       # [1, L-1]
                dec_labels    = gold_tokens[:,  1:]       # [1, L-1]
            else:
                # fast path: assume tensors already provided
                assert dec_input_ids is not None and dec_labels is not None, \
                    "When using enc_input_ids, you must also pass dec_input_ids and dec_labels"

            pad_id = tokenizer.pad_token_id
            dec_attn_mask = (dec_input_ids != pad_id).long()                      # [B, T]
            enc_attn_mask = torch.ones(personalized_enc.size()[:2],               # [B, L]
                                       device=self.device, dtype=torch.long)

            # Decoder forward
            outputs = self.t5.decoder(
                input_ids=dec_input_ids,
                attention_mask=dec_attn_mask,
                encoder_hidden_states=personalized_enc,
                encoder_attention_mask=enc_attn_mask
            )
            lm_logits = self.t5.lm_head(outputs.last_hidden_state)  # [B, T, V]

            # Safety (no hard clamp so grads flow)
            if not torch.isfinite(lm_logits).all():
                lm_logits = torch.nan_to_num(lm_logits, nan=0.0, posinf=1e4, neginf=-1e4)

            gen_loss_fct = nn.CrossEntropyLoss(ignore_index=pad_id)
            gen_loss = gen_loss_fct(
                lm_logits.view(-1, lm_logits.size(-1)),
                dec_labels.view(-1)
            )

            # Let CE drive gradients; enc_loss detached for stability at start
            total_loss = 0.4 * gen_loss + 0.6 * enc_loss

            # Optional: cheap preview ONLY every preview_stride steps
            do_preview = (preview_stride and global_step is not None and global_step % preview_stride == 0)
            pred_summary_tf = ""
            if do_preview:
                with torch.no_grad():
                    # Build minimal greedy preview using the same personalized_enc
                    start_id = getattr(self.t5.config, "decoder_start_token_id", None)
                    if start_id is None:
                        start_id = getattr(tokenizer, "pad_token_id", None)
                    if start_id is None:
                        start_id = getattr(tokenizer, "eos_token_id", 1)
                    eos_id = getattr(tokenizer, "eos_token_id", None)
                    decoder_input = torch.tensor([[start_id]], device=self.device)
                    enc_mask = enc_attn_mask[:1]  # [1,L]
                    for _ in range(max(1, dec_labels.size(1))):
                        dec_out = self.t5.decoder(
                            input_ids=decoder_input,
                            encoder_hidden_states=personalized_enc[:1],
                            encoder_attention_mask=enc_mask
                        )
                        step_logits = self.t5.lm_head(dec_out.last_hidden_state)[:, -1, :]
                        step_logits = torch.clamp(step_logits, -50, 50)
                        next_id = torch.argmax(step_logits, dim=-1, keepdim=True)
                        decoder_input = torch.cat([decoder_input, next_id], dim=1)
                        if eos_id is not None and next_id.item() == eos_id:
                            break
                    pred_tokens = decoder_input[0, 1:]
                    pred_summary_tf = tokenizer.decode(pred_tokens, skip_special_tokens=True)

            return {
                "skip": False,
                "total_loss": total_loss,
                "enc_loss": enc_loss,
                "gen_loss": gen_loss,
                "lm_logits": lm_logits,
                "doc_text": doc_text,
                "gold_summary": gold_summary_text,
                "pred_summary_tf": pred_summary_tf,
            }

        # -------------------------------
        # Autoregressive (evaluation)
        # -------------------------------
        elif mode in ("eval", "test", "ar", "greedy", "beam"):
            enc_len = personalized_enc.size(1)
            enc_mask = torch.ones((personalized_enc.size(0), enc_len), dtype=torch.long, device=self.device)

            # pick start/eos ids safely
            start_id = getattr(self.t5.config, "decoder_start_token_id", None)
            if start_id is None:
                start_id = getattr(tokenizer, "pad_token_id", None)
            if start_id is None:
                start_id = getattr(tokenizer, "eos_token_id", 1)
            eos_id = getattr(tokenizer, "eos_token_id", None)

            # Start token
            decoder_input = torch.tensor([[start_id]], device=self.device)  # [1, 1]

            # We will generate up to max_len-1 new tokens
            max_new_tokens = max_len - 1

            with torch.no_grad():
                for _ in range(max_new_tokens):
                    dec_out = self.t5.decoder(
                        input_ids=decoder_input,
                        encoder_hidden_states=personalized_enc[:1],
                        encoder_attention_mask=enc_mask[:1]
                    )
                    lm_logits = self.t5.lm_head(dec_out.last_hidden_state)  # [1, T, V]
                    next_token_logits = lm_logits[:, -1, :]                  # [1, V]
                    next_token_logits = torch.clamp(next_token_logits, -50, 50)
                    next_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)  # [1, 1]

                    decoder_input = torch.cat([decoder_input, next_id], dim=1)

                    if eos_id is not None and next_id.item() == eos_id:
                        break

            pred_tokens = decoder_input[0, 1:]
            pred_summary_ar = tokenizer.decode(pred_tokens, skip_special_tokens=True)

            return {
                "pred_summary_ar": pred_summary_ar,
                "enc_loss": enc_loss,
                "doc_text": doc_text,
                "gold_summary": gold_summary_text,
            }


In [12]:
# =========================
# Forward pass + Encoder Top-10 & Metrics (AUC/MRR/nDCG@5)
# =========================
import os, re, math, torch, numpy as np
import torch.nn.functional as F
from transformers.modeling_outputs import BaseModelOutput

# ---------- helper: latest ckpt ----------
def find_latest_ckpt(ckpt_dir="checkpoints"):
    paths = [os.path.join(ckpt_dir, f) for f in os.listdir(ckpt_dir)
             if f.startswith("epoch_") and f.endswith(".pt")]
    if not paths:
        raise FileNotFoundError(f"No checkpoints found in {ckpt_dir}")
    def _ep(p):
        m = re.search(r"epoch_(\d+)\.pt$", os.path.basename(p))
        return int(m.group(1)) if m else -1
    paths.sort(key=lambda p: (_ep(p), os.path.getmtime(p)))
    return paths[-1]

# ---------- 0) Load checkpoint & build model ----------
latest_ckpt = find_latest_ckpt("checkpoints")
ckpt = torch.load(latest_ckpt, map_location=device)
print(f"🔎 Using checkpoint: {latest_ckpt}")
print(f"   epoch={ckpt.get('epoch')} | avg_total={ckpt.get('avg_total_loss')} | enc={ckpt.get('avg_enc_loss')} | gen={ckpt.get('avg_gen_loss')}")

behavior_encoder = BehaviorEncoder(hidden_dim, device=device, max_len=max_len).to(device)
inverse_decoder  = BehaviorInverseDecoderPredict(hidden_dim, device=device).to(device)

personalized_model = PersonalizedT5Summarizer(
    hidden_dim, summarizer_model, behavior_encoder, inverse_decoder, device,
    learnable_ctx=True, unfreeze_last_decoder_blocks=15
).to(device)

missing, unexpected = personalized_model.load_state_dict(ckpt["model_state"], strict=False)
print(f"ℹ️ load_state: missing={len(missing)} unexpected={len(unexpected)}")

# ---------- 1) Sample 10 rows ----------
batch_rows = train_df.sample(10, random_state=42)
Bhist_batch = batch_rows["EHist_padded"].tolist()
Bhist_lens  = batch_rows["EHist_len"].tolist()
Bpos_batch  = batch_rows["EPos"].tolist()
print(f"🔹 Sampled batch size: {len(Bhist_batch)}")

# ---------- 2) TRAIN pass (TF) + preview & token IDs (unchanged) ----------
personalized_model.train()
tf_total_losses, tf_enc_losses, tf_gen_losses = [], [], []
tf_pred_texts, tf_doc_texts, tf_gold_texts = [], [], []
tf_gold_ids, tf_pred_ids = [], []

for i in range(len(Bhist_batch)):
    Bhist     = Bhist_batch[i]
    Bhist_len = int(Bhist_lens[i])
    Bpos      = Bpos_batch[i]

    out = personalized_model(
        Bhist, Bhist_len, Bpos,
        lookup_df, embed_tables, sid2sum, nid2body, tokenizer,
        max_len=50, mode="train",
        preview_stride=1, global_step=i
    )

    tf_total_losses.append(out["total_loss"].item())
    tf_enc_losses.append(out["enc_loss"].item())
    tf_gen_losses.append(out["gen_loss"].item())
    tf_pred_texts.append(out["pred_summary_tf"])
    tf_doc_texts.append(out["doc_text"])
    tf_gold_texts.append(out["gold_summary"])

    if out["lm_logits"] is not None:
        pred_ids = torch.argmax(out["lm_logits"], dim=-1)  # [B, T]
        tf_pred_ids.append(pred_ids[0].tolist())
        gold_tokens = tokenizer(
            out["gold_summary"],
            return_tensors="pt",
            truncation=True,
            max_length=50,
            padding="max_length"
        ).input_ids.to(device)
        tf_gold_ids.append(gold_tokens[0].tolist())
    else:
        tf_pred_ids.append([])
        tf_gold_ids.append([])

print("\n✅ Teacher-Forcing (TRAIN) Results")
for i in range(len(Bhist_batch)):
    print(f"\n--- Sample {i} ---")
    print(f"Doc:  {tf_doc_texts[i][:200]}")
    print(f"Gold: {tf_gold_texts[i][:200]}")
    print(f"TF Pred: {tf_pred_texts[i][:200]}")
    print(f"Loss={tf_total_losses[i]:.4f} | Enc={tf_enc_losses[i]:.4f} | Gen={tf_gen_losses[i]:.4f}")
    print(f"Gold IDs: {tf_gold_ids[i]}")
    print(f"Pred IDs: {tf_pred_ids[i]}")
    if tf_gold_ids[i]:
        print("Gold decoded:", tokenizer.decode(tf_gold_ids[i], skip_special_tokens=True))
    if tf_pred_ids[i]:
        print("Pred decoded:", tokenizer.decode(tf_pred_ids[i], skip_special_tokens=True))

# ---------- 2.5) BehaviorEncoder Top-10 & Metrics ----------
print("\n🧭 Encoder Next-Step Prediction (Top-10) & Metrics")
all_ranks, all_auc, all_mrr, all_ndcg5 = [], [], [], []

def safe_softmax(x):
    if not torch.isfinite(x).all():
        x = torch.nan_to_num(x, nan=-1e4, posinf=1e4, neginf=-1e4)
    return F.softmax(x, dim=-1)

for i in range(len(Bhist_batch)):
    Bhist     = Bhist_batch[i]
    Bhist_len = int(Bhist_lens[i])   # ground-truth next position (index of next step)
    Bpos      = Bpos_batch[i]

    with torch.no_grad():
        _, e_next, logits_pos, _, _ = behavior_encoder(
            Bhist, Bhist_len, Bpos, lookup_df, embed_tables
        )  # logits_pos: [1, C]
        probs = safe_softmax(logits_pos.squeeze(0))  # [C]
        C = probs.numel()

        # top-10 predictions
        K = min(10, C)
        vals, idxs = torch.topk(probs, k=K)
        vals = vals.detach().cpu().numpy()
        idxs = idxs.detach().cpu().numpy()

        # rank of the true target
        sorted_all = torch.argsort(probs, descending=True).detach().cpu().numpy().tolist()
        if Bhist_len < 0 or Bhist_len >= C:
            rank = C  # invalid target -> worst
        else:
            rank = sorted_all.index(Bhist_len) + 1  # 1-based rank

        # Metrics
        auc_i = ((C - rank) / (C - 1)) if C > 1 else 1.0
        mrr_i = 1.0 / rank
        ndcg5_i = (1.0 / math.log2(1 + rank)) if rank <= 5 else 0.0

        all_ranks.append(rank)
        all_auc.append(auc_i)
        all_mrr.append(mrr_i)
        all_ndcg5.append(ndcg5_i)

    # Pretty print
    print(f"\n— Sample {i} — target_pos={Bhist_len} | rank={rank} | AUC={auc_i:.3f} | MRR={mrr_i:.3f} | nDCG@5={ndcg5_i:.3f}")
    for k in range(K):
        marker = "✅" if idxs[k] == Bhist_len else "  "
        print(f"  {marker} #{k+1:>2}: pos={int(idxs[k]):>3}  p={vals[k]:.4f}")

def _mean(x):
    return float(np.mean(x)) if len(x)>0 else float('nan')

print("\n📊 Batch Metrics (encoder next-step)")
print(f"  mean-rank = {_mean(all_ranks):.2f}")
print(f"  AUC       = {_mean(all_auc):.3f}")
print(f"  MRR       = {_mean(all_mrr):.3f}")
print(f"  nDCG@5    = {_mean(all_ndcg5):.3f}")

# ---------- 3) EVAL pass (AR) — simple greedy w/out KV cache ----------
personalized_model.eval()
ar_pred_texts, ar_doc_texts, ar_gold_texts, ar_enc_losses = [], [], [], []
ar_gold_ids, ar_pred_ids = [], []

with torch.no_grad():
    for i in range(len(Bhist_batch)):
        Bhist     = Bhist_batch[i]
        Bhist_len = int(Bhist_lens[i])
        Bpos      = Bpos_batch[i]

        # behavior + encoder enc
        e_last, s_hat, enc_loss = personalized_model._behavior_encode(
            Bhist, Bhist_len, Bpos, lookup_df, embed_tables
        )
        base_enc, doc_text, tail_id = personalized_model._encode_query(
            Bpos, lookup_df, nid2body, tokenizer, max_len=50
        )
        if base_enc.dim() == 2:
            base_enc = base_enc.unsqueeze(0)

        personalized_enc = personalized_model._inject_context(base_enc, e_last, s_hat)
        if not torch.isfinite(personalized_enc).all():
            personalized_enc = torch.nan_to_num(personalized_enc, nan=0.0, posinf=0.0, neginf=0.0)

        # encoder attention mask derived from same tokens
        query_tokens = tokenizer(
            "Generate headline: " + doc_text,
            return_tensors="pt", truncation=True,
            max_length=50, padding="max_length"
        ).input_ids.to(device)
        enc_attn_mask = (query_tokens != tokenizer.pad_token_id).long()

        # greedy decode (no cache; simplest + robust across HF versions)
        pad_id = tokenizer.pad_token_id
        eos_id = tokenizer.eos_token_id
        start_id = personalized_model.t5.config.decoder_start_token_id or pad_id

        dec = torch.tensor([[start_id]], device=device, dtype=torch.long)  # [1,1]
        max_new, min_new = 49, 5
        for _ in range(max_new):
            dec_out = personalized_model.t5.decoder(
                input_ids=dec,
                encoder_hidden_states=personalized_enc,
                encoder_attention_mask=enc_attn_mask,
                use_cache=False,
                return_dict=True,
            )
            logits = personalized_model.t5.lm_head(dec_out.last_hidden_state)  # [1,T,V]
            next_logits = logits[:, -1, :]
            # block PAD; delay EOS
            if pad_id is not None:
                next_logits[:, pad_id] = -1e9
            if eos_id is not None and dec.size(1) - 1 < min_new:
                next_logits[:, eos_id] = -1e9
            next_id = torch.argmax(next_logits, dim=-1, keepdim=True)
            dec = torch.cat([dec, next_id], dim=1)
            if eos_id is not None and next_id.item() == eos_id:
                break

        pred_tokens = dec[0, 1:]
        pred_summary_ar = tokenizer.decode(pred_tokens, skip_special_tokens=True)

        # gold ids
        gold_text = sid2sum.get(tail_id, "")
        gold_ids = tokenizer(
            gold_text,
            return_tensors="pt",
            truncation=True,
            max_length=50,
            padding="max_length"
        ).input_ids[0]

        ar_pred_texts.append(pred_summary_ar)
        ar_doc_texts.append(doc_text)
        ar_gold_texts.append(gold_text)
        ar_enc_losses.append(enc_loss.item() if torch.is_tensor(enc_loss) else float(enc_loss))
        ar_gold_ids.append(gold_ids.tolist())
        ar_pred_ids.append(pred_tokens.tolist())

print("\n✅ Autoregressive (EVAL) Results")
for i in range(len(Bhist_batch)):
    print(f"\n--- Sample {i} ---")
    print(f"Doc:  {ar_doc_texts[i][:200]}")
    print(f"Gold: {ar_gold_texts[i][:200]}")
    print(f"AR Pred: {ar_pred_texts[i][:200]}")
    print(f"EncLoss={ar_enc_losses[i]:.4f}")
    print(f"Gold IDs: {ar_gold_ids[i]}")
    print(f"Pred IDs: {ar_pred_ids[i]}")
    if ar_gold_ids[i]:
        print("Gold decoded:", tokenizer.decode(ar_gold_ids[i], skip_special_tokens=True))
    if ar_pred_ids[i]:
        print("Pred decoded:", tokenizer.decode(ar_pred_ids[i], skip_special_tokens=True))


🔎 Using checkpoint: checkpoints/epoch_5.pt
   epoch=5 | avg_total=2.014425294696192 | enc=2.9589636535152124 | gen=0.5976175675267787
ℹ️ load_state: missing=0 unexpected=0
🔹 Sampled batch size: 10

✅ Teacher-Forcing (TRAIN) Results

--- Sample 0 ---
Doc:  A tabloid claims Jennifer Lopez won't marry Alex Rodriguez unless he signs a strict prenup with a cheating clause, and it's causing a major fight between the couple. The story is completely untrue. Go
Gold: Early retirement can be more than just a daydream for those long Tuesday afternoons at work.
TF Pred: The prince and his wife post  topically: How they came together
Loss=3.0175 | Enc=3.4529 | Gen=2.3643
Gold IDs: [8840, 6576, 54, 36, 72, 145, 131, 3, 9, 239, 26066, 21, 273, 307, 2818, 3742, 7, 44, 161, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Pred IDs: [3957, 16, 36, 1137, 145, 3, 3, 9, 710, 26066, 21, 128, 113, 307, 1379, 7, 5, 161, 5, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1

In [13]:
# ---- Param report helpers ----
from collections import defaultdict
import math

def count_params(module, only_trainable=False):
    return sum(p.numel() for p in module.parameters() if (p.requires_grad or not only_trainable))

def param_table_by_prefix(model, topn=25):
    buckets = defaultdict(int)
    for n, p in model.named_parameters():
        if not p.requires_grad:
            continue
        prefix = ".".join(n.split(".")[:4])
        buckets[prefix] += p.numel()
    return sorted(buckets.items(), key=lambda x: x[1], reverse=True)[:topn]

def print_param_report(model):
    total = count_params(model, only_trainable=False)
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen = total - trainable
    print(f"📦 Total params: {total:,}")
    print(f"🟢 Trainable:   {trainable:,}")
    print(f"🧊 Frozen:      {frozen:,}")

    # show breakdown of where the trainable params live
    rows = param_table_by_prefix(model, topn=40)
    print("\nTop trainable parameter groups:")
    for k, v in rows:
        print(f"  • {k:60s} {v:,}")

print_param_report(personalized_model)

# sanity: list a few key modules that must be trainable
must_be_trainable = [
    "t5.shared.weight", "t5.lm_head.weight",
    "t5.decoder.final_layer_norm.weight",
    # last N decoder blocks (you set N in the class' __init__)
    "t5.decoder.block.-1.layer.0.SelfAttention.q.weight"  # won't match literally; just a reminder
]
print("\nQuick spot-check on a few important params:")
for name, p in personalized_model.named_parameters():
    if any(k in name for k in ["t5.shared.weight", "t5.lm_head.weight", "t5.decoder.final_layer_norm.weight"]):
        print(f"  • {name:60s} requires_grad={p.requires_grad}")


📦 Total params: 232,068,503
🟢 Trainable:   147,114,263
🧊 Frozen:      84,954,240

Top trainable parameter groups:
  • t5.shared.weight                                             24,674,304
  • t5.decoder.block.0                                           9,439,872
  • t5.decoder.block.1                                           9,439,488
  • t5.decoder.block.2                                           9,439,488
  • t5.decoder.block.3                                           9,439,488
  • t5.decoder.block.4                                           9,439,488
  • t5.decoder.block.5                                           9,439,488
  • t5.decoder.block.6                                           9,439,488
  • t5.decoder.block.7                                           9,439,488
  • t5.decoder.block.8                                           9,439,488
  • t5.decoder.block.9                                           9,439,488
  • t5.decoder.block.10                                     

In [14]:
from torch.utils.data import Dataset, DataLoader

class NewsDataset(Dataset):
    def __init__(self, df):
        # df must contain: EHist_padded, EHist_len, EPos
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return {
            "Bhist": row["EHist_padded"],
            "Bhist_len": int(row["EHist_len"]),
            "Bpos": row["EPos"],
        }

def collate_simple(batch):
    # keep as lists; model forward works per-sample
    Bhist = [b["Bhist"] for b in batch]
    Blen  = [b["Bhist_len"] for b in batch]
    Bpos  = [b["Bpos"] for b in batch]
    return Bhist, Blen, Bpos

# build dataset / dataloader
train_ds = NewsDataset(train_df)
train_loader = DataLoader(
    train_ds,
    batch_size=8,          # tune as your GPU allows
    shuffle=True,
    num_workers=0,
    pin_memory=False,
    collate_fn=collate_simple
)

print(f"✅ DataLoader ready: {len(train_ds)} samples, batch_size={train_loader.batch_size}")


✅ DataLoader ready: 38417 samples, batch_size=8


In [15]:
def grad_health_report(model, limit_print=30):
    issues = {"none": [], "all_zero": [], "nan_inf": []}
    for name, p in model.named_parameters():
        if not p.requires_grad:
            continue
        g = p.grad
        if g is None:
            issues["none"].append(name)
            continue
        if not torch.isfinite(g).all():
            issues["nan_inf"].append(name)
            continue
        if torch.count_nonzero(g).item() == 0:
            issues["all_zero"].append(name)

    total_bad = sum(len(v) for v in issues.values())
    if total_bad == 0:
        print("👍 gradients look healthy on all trainable params.")
        return True

    print(f"⚠️  {total_bad} gradient issues:")
    for k in ["none", "all_zero", "nan_inf"]:
        if issues[k]:
            shown = issues[k][:limit_print]
            more = len(issues[k]) - len(shown)
            print(f"  • {k}: {len(issues[k])} {'(showing first ' + str(len(shown)) + ')' if more >= 0 else ''}")
            for n in shown:
                print(f"     - {n}")
            if more > 0:
                print(f"     ... and {more} more")
    return False


In [ ]:
# import os, re, torch
# from tqdm import tqdm
# import torch.optim as optim

# def find_latest_ckpt(ckpt_dir="checkpoints"):
#     """Return path of latest checkpoint in a folder, or None if none exist."""
#     if not os.path.isdir(ckpt_dir):
#         return None
#     paths = [os.path.join(ckpt_dir, f) for f in os.listdir(ckpt_dir)
#              if f.startswith("epoch_") and f.endswith(".pt")]
#     if not paths:
#         return None
#     def _ep(p):
#         m = re.search(r"epoch_(\d+)\.pt$", os.path.basename(p))
#         return int(m.group(1)) if m else -1
#     return max(paths, key=lambda p: _ep(p))

# # --- optimizer over trainable params only ---
# trainable_params = [p for p in personalized_model.parameters() if p.requires_grad]
# optimizer = optim.AdamW(trainable_params, lr=2e-4, weight_decay=0.01)

# epochs = 5          # total training epochs (set as you like)
# max_gen_len = 50
# clip_norm = 1.0
# os.makedirs("checkpoints", exist_ok=True)

# # ==== RESUME OR START FRESH ====
# start_epoch = 1
# latest_ckpt = find_latest_ckpt("checkpoints")
# if latest_ckpt:
#     ckpt = torch.load(latest_ckpt, map_location=device)
#     print(f"🔎 Found checkpoint: {latest_ckpt}")
    
#     # check param counts
#     ckpt_params = len(ckpt["model_state"])
#     model_params = len(personalized_model.state_dict())
#     print(f"📊 Checkpoint params={ckpt_params}, Current model params={model_params}")
    
#     # load weights
#     personalized_model.load_state_dict(ckpt["model_state"], strict=False)
#     optimizer.load_state_dict(ckpt["optimizer_state"])
#     personalized_model.to(device)
    
#     last_epoch = ckpt["epoch"]
#     print(f"✅ Resuming from epoch {last_epoch}, "
#           f"last losses: tot={ckpt.get('avg_total_loss'):.4f}, "
#           f"enc={ckpt.get('avg_enc_loss'):.4f}, "
#           f"gen={ckpt.get('avg_gen_loss'):.4f}")
#     start_epoch = last_epoch + 1
# else:
#     print("⚠️ No checkpoint found, starting fresh training.")
#     personalized_model.to(device)

# # ==== TRAINING LOOP ====
# for ep in range(start_epoch, epochs + 1):
#     personalized_model.train()
#     pbar = tqdm(train_loader, desc=f"Epoch {ep}/{epochs}", dynamic_ncols=True)
#     running_tot, running_enc, running_gen = 0.0, 0.0, 0.0
#     seen = 0

#     for Bhist_batch, Blen_batch, Bpos_batch in pbar:
#         optimizer.zero_grad(set_to_none=True)

#         batch_total, batch_enc, batch_gen = 0.0, 0.0, 0.0
#         valid_in_batch = 0

#         for i in range(len(Bhist_batch)):
#             out = personalized_model(
#                 Bhist_batch[i], int(Blen_batch[i]), Bpos_batch[i],
#                 lookup_df, embed_tables, sid2sum, nid2body, tokenizer,
#                 max_len=max_gen_len, mode="train"
#             )
#             if out["skip"]:
#                 continue

#             total_loss = out["total_loss"]
#             gen_loss   = out["gen_loss"]
#             enc_loss   = out["enc_loss"]

#             batch_total += float(total_loss.detach().item())
#             batch_enc   += float(enc_loss.detach().item()) if torch.is_tensor(enc_loss) else float(enc_loss)
#             batch_gen   += float(gen_loss.detach().item())
#             valid_in_batch += 1

#             total_loss.backward()

#         if valid_in_batch == 0:
#             pbar.set_postfix_str("skip batch")
#             continue

#         torch.nn.utils.clip_grad_norm_(trainable_params, clip_norm)
#         optimizer.step()

#         ok = all(
#             torch.isfinite(p.grad).all()
#             for _, p in personalized_model.named_parameters()
#             if p.requires_grad and p.grad is not None
#         )

#         running_tot += batch_total
#         running_enc += batch_enc
#         running_gen += batch_gen
#         seen        += valid_in_batch

#         avg_tot = running_tot / max(1, seen)
#         avg_enc = running_enc / max(1, seen)
#         avg_gen = running_gen / max(1, seen)

#         pbar.set_postfix({
#             "enc": f"{avg_enc:.3f}",
#             "dec": f"{avg_gen:.3f}",
#             "tot": f"{avg_tot:.3f}",
#             "grad": "👍" if ok else "⚠️"
#         })

#     # ---- SAVE CHECKPOINT AT END OF EPOCH ----
#     ckpt_path = f"checkpoints/epoch_{ep}.pt"
#     torch.save({
#         "epoch": ep,
#         "model_state": personalized_model.state_dict(),
#         "optimizer_state": optimizer.state_dict(),
#         "avg_total_loss": avg_tot,
#         "avg_enc_loss": avg_enc,
#         "avg_gen_loss": avg_gen,
#     }, ckpt_path)
#     print(f"💾 Saved checkpoint: {ckpt_path}")

#     # optional grad health report
#     print("\nEpoch grad health summary:")
#     grad_health_report(personalized_model, limit_print=40)


In [20]:
# =========================
# Encoder-only training cell (freezes everything else)
# =========================
import os, re, torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ---------- helpers ----------
def print_param_report(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"📦 Total params: {total:,}")
    print(f"🟢 Trainable:   {trainable:,}")
    print(f"🧊 Frozen:      {total-trainable:,}")

def grad_health_report(model, limit_print=30):
    issues = {"none": [], "all_zero": [], "nan_inf": []}
    for name, p in model.named_parameters():
        if not p.requires_grad:
            continue
        g = p.grad
        if g is None:
            issues["none"].append(name); continue
        if not torch.isfinite(g).all():
            issues["nan_inf"].append(name); continue
        if torch.count_nonzero(g).item() == 0:
            issues["all_zero"].append(name)
    total_bad = sum(len(v) for v in issues.values())
    if total_bad == 0:
        print("👍 gradients look healthy on all trainable params.")
        return True
    print(f"⚠️  {total_bad} gradient issues:")
    for k in ["none", "all_zero", "nan_inf"]:
        if issues[k]:
            shown = issues[k][:limit_print]
            more = len(issues[k]) - len(shown)
            print(f"  • {k}: {len(issues[k])}")
            for n in shown: print(f"     - {n}")
            if more > 0: print(f"     ... and {more} more")
    return False

def find_latest_ckpt(ckpt_dir="checkpoints"):
    if not os.path.isdir(ckpt_dir): return None
    paths = [os.path.join(ckpt_dir, f) for f in os.listdir(ckpt_dir)
             if f.startswith("epoch_") and f.endswith(".pt")]
    if not paths: return None
    def _ep(p):
        m = re.search(r"epoch_(\d+)\.pt$", os.path.basename(p))
        return int(m.group(1)) if m else -1
    return max(paths, key=lambda p: _ep(p))

# ---------- dataset/dataloader ----------
class NewsDataset(Dataset):
    def __init__(self, df):
        # df must contain: EHist_padded, EHist_len, EPos
        self.df = df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        return {"Bhist": r["EHist_padded"], "Bhist_len": int(r["EHist_len"]), "Bpos": r["EPos"]}

def collate_simple(batch):
    return (
        [b["Bhist"] for b in batch],
        [b["Bhist_len"] for b in batch],
        [b["Bpos"] for b in batch],
    )

train_ds = NewsDataset(train_df)
train_loader = DataLoader(
    train_ds, batch_size=8, shuffle=True, num_workers=0, pin_memory=False, collate_fn=collate_simple
)
print(f"✅ DataLoader: {len(train_ds)} samples, batch_size={train_loader.batch_size}")

# ---------- build model (fresh) ----------
behavior_encoder = BehaviorEncoder(hidden_dim, device=device, max_len=max_len).to(device)
inverse_decoder  = BehaviorInverseDecoderPredict(hidden_dim, device=device).to(device)

personalized_model = PersonalizedT5Summarizer(
    hidden_dim, summarizer_model, behavior_encoder, inverse_decoder, device,
    learnable_ctx=True, unfreeze_last_decoder_blocks=0  # <- do NOT unfreeze any T5 decoder blocks
).to(device)

# ---------- freeze everything except behavior_encoder ----------
for name, p in personalized_model.named_parameters():
    p.requires_grad = name.startswith("behavior_encoder")

print_param_report(personalized_model)

# ---------- optimizer over encoder params only ----------
enc_params = [p for p in personalized_model.behavior_encoder.parameters() if p.requires_grad]
assert len(enc_params) > 0, "No trainable params found in behavior_encoder!"
optimizer = optim.AdamW(enc_params, lr=2e-4, weight_decay=0.01)

# ---------- (optional) resume ----------
os.makedirs("checkpoints", exist_ok=True)
start_epoch = 1
latest = find_latest_ckpt("checkpoints")
if latest:
    ckpt = torch.load(latest, map_location=device)
    print(f"🔎 Found checkpoint: {latest}")
    # Safely load model (shape changes tolerated)
    personalized_model.load_state_dict(ckpt["model_state"], strict=False)
    # Optimizer may fail to load if param set changed — guard it:
    if "optimizer_state" in ckpt:
        try:
            optimizer.load_state_dict(ckpt["optimizer_state"])
        except Exception as e:
            print(f"⚠️ Skipping optimizer_state load: {e}")
    last_epoch = int(ckpt.get("epoch", 0))
    print(f"✅ Resuming from epoch {last_epoch}; prev avg_enc_loss={ckpt.get('avg_enc_loss')}")
    start_epoch = last_epoch + 1
else:
    print("▶️ Starting fresh encoder-only training.")

# ---------- train (encoder-only) ----------
epochs =8
clip_norm = 1.0

for ep in range(start_epoch, epochs + 1):
    personalized_model.train()
    pbar = tqdm(train_loader, desc=f"[ENC-ONLY] Epoch {ep}/{epochs}", dynamic_ncols=True)
    running_enc, seen = 0.0, 0

    for Bhist_batch, Blen_batch, Bpos_batch in pbar:
        optimizer.zero_grad(set_to_none=True)

        batch_enc, valid = 0.0, 0
        for i in range(len(Bhist_batch)):
            # Forward the behavior encoder ONLY; use its total loss
            e_last, e_next, logits_pos, ctx_enc_loss, enc_total_loss = personalized_model.behavior_encoder(
                Bhist_batch[i], int(Blen_batch[i]), Bpos_batch[i], lookup_df, embed_tables
            )
            loss = enc_total_loss
            loss.backward()
            batch_enc += float(loss.detach().item())
            valid += 1

        if valid == 0:
            pbar.set_postfix_str("skip batch"); continue

        torch.nn.utils.clip_grad_norm_(enc_params, clip_norm)
        optimizer.step()

        running_enc += batch_enc
        seen += valid
        avg_enc = running_enc / max(1, seen)
        pbar.set_postfix({"enc": f"{avg_enc:.3f}"})

    # ---- save checkpoint ----
    ckpt_path = f"checkpoints/epoch_{ep}.pt"
    torch.save({
        "epoch": ep,
        "model_state": personalized_model.state_dict(),   # includes encoder weights
        "optimizer_state": optimizer.state_dict(),
        "avg_total_loss": None,
        "avg_enc_loss": avg_enc,
        "avg_gen_loss": None,
    }, ckpt_path)
    print(f"💾 Saved encoder-only checkpoint: {ckpt_path}")

    print("\nEpoch grad health summary (encoder only):")
    grad_health_report(personalized_model, limit_print=40)


✅ DataLoader: 38417 samples, batch_size=8
📦 Total params: 232,068,503
🟢 Trainable:   5,033,110
🧊 Frozen:      227,035,393


🔎 Found checkpoint: checkpoints/epoch_5.pt
⚠️ Skipping optimizer_state load: loaded state dict contains a parameter group that doesn't match the size of optimizer's group
✅ Resuming from epoch 5; prev avg_enc_loss=2.9589636535152124


[ENC-ONLY] Epoch 6/8:   0%|          | 4/4803 [00:08<2:41:14,  2.02s/it, enc=2.852]


KeyboardInterrupt: 

In [21]:
# =========================
# Train ONLY: inverse_decoder + personalized summarizer (T5 decoder + injectors)
# =========================
import os, re, torch, math
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ---------- helpers ----------
def print_param_report(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"📦 Total params: {total:,}")
    print(f"🟢 Trainable:   {trainable:,}")
    print(f"🧊 Frozen:      {total-trainable:,}")

def grad_health_report(model, limit_print=30):
    issues = {"none": [], "all_zero": [], "nan_inf": []}
    for name, p in model.named_parameters():
        if not p.requires_grad:
            continue
        g = p.grad
        if g is None: issues["none"].append(name); continue
        if not torch.isfinite(g).all(): issues["nan_inf"].append(name); continue
        if torch.count_nonzero(g).item() == 0: issues["all_zero"].append(name)
    total_bad = sum(len(v) for v in issues.values())
    if total_bad == 0:
        print("👍 gradients look healthy on all trainable params.")
        return True
    print(f"⚠️  {total_bad} gradient issues:")
    for k in ["none", "all_zero", "nan_inf"]:
        if issues[k]:
            shown = issues[k][:limit_print]
            more = len(issues[k]) - len(shown)
            print(f"  • {k}: {len(issues[k])}")
            for n in shown: print(f"     - {n}")
            if more > 0: print(f"     ... and {more} more")
    return False

def find_latest_ckpt(ckpt_dir="checkpoints"):
    if not os.path.isdir(ckpt_dir): return None
    paths = [os.path.join(ckpt_dir, f) for f in os.listdir(ckpt_dir)
             if f.startswith("epoch_") and f.endswith(".pt")]
    if not paths: return None
    def _ep(p):
        m = re.search(r"epoch_(\d+)\.pt$", os.path.basename(p))
        return int(m.group(1)) if m else -1
    return max(paths, key=lambda p: _ep(p))

# ---------- dataset/dataloader ----------
class NewsDataset(Dataset):
    def __init__(self, df):
        # df must contain: EHist_padded, EHist_len, EPos
        self.df = df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        return {"Bhist": r["EHist_padded"], "Bhist_len": int(r["EHist_len"]), "Bpos": r["EPos"]}

def collate_simple(batch):
    return (
        [b["Bhist"] for b in batch],
        [b["Bhist_len"] for b in batch],
        [b["Bpos"] for b in batch],
    )

train_ds = NewsDataset(train_df)
train_loader = DataLoader(
    train_ds, batch_size=8, shuffle=True, num_workers=0, pin_memory=False, collate_fn=collate_simple
)
print(f"✅ DataLoader: {len(train_ds)} samples, batch_size={train_loader.batch_size}")

# ---------- build model (fresh) ----------
# Unfreeze some last T5 decoder blocks so they can learn (tune N as you like)
UNFREEZE_LAST_DECODER_BLOCKS = 12

behavior_encoder = BehaviorEncoder(hidden_dim, device=device, max_len=max_len).to(device)
inverse_decoder  = BehaviorInverseDecoderPredict(hidden_dim, device=device).to(device)

personalized_model = PersonalizedT5Summarizer(
    hidden_dim, summarizer_model, behavior_encoder, inverse_decoder, device,
    learnable_ctx=True, unfreeze_last_decoder_blocks=UNFREEZE_LAST_DECODER_BLOCKS
).to(device)

# ---------- freeze everything, then selectively enable trainable parts ----------
for name, p in personalized_model.named_parameters():
    p.requires_grad = False

def allow(name: str) -> bool:
    # train inverse_decoder
    if name.startswith("inverse_decoder."): return True
    # train context-injection projections
    if name.startswith("e_proj.") or name.startswith("q_proj.") or name.startswith("k_proj.") or name.startswith("v_proj."): return True
    if name == "ctx_scale_raw": return True
    # train T5 decoder last N blocks (already unfrozen in __init__, but we enforce here)
    if name.startswith("t5.decoder.block."): return True
    # train decoder final layer norm
    if name.startswith("t5.decoder.final_layer_norm."): return True
    # train tied embeddings + lm_head (T5 weight tying)
    if name.startswith("t5.shared.") or name.startswith("t5.lm_head."): return True
    return False

for name, p in personalized_model.named_parameters():
    if allow(name):
        p.requires_grad = True

# explicitly ensure encoder is frozen
for name, p in personalized_model.named_parameters():
    if name.startswith("behavior_encoder."):
        p.requires_grad = False

print_param_report(personalized_model)

# ---------- optimizer over trainable params (inverse+summarizer only) ----------
trainable_params = [p for p in personalized_model.parameters() if p.requires_grad]
assert len(trainable_params) > 0, "No trainable params selected!"
optimizer = optim.AdamW(trainable_params, lr=1.5e-4, weight_decay=0.01)  # tune LR as needed

# ---------- (optional) resume ----------
os.makedirs("checkpoints", exist_ok=True)
start_epoch = 1
latest = find_latest_ckpt("checkpoints")
if latest:
    ckpt = torch.load(latest, map_location=device)
    print(f"🔎 Found checkpoint: {latest}")
    # Load weights permissively (shape changes tolerated)
    personalized_model.load_state_dict(ckpt["model_state"], strict=False)
    # Optimizer might not match param groups if you previously trained different parts
    if "optimizer_state" in ckpt:
        try:
            optimizer.load_state_dict(ckpt["optimizer_state"])
        except Exception as e:
            print(f"⚠️ Skipping optimizer_state load: {e}")
    last_epoch = int(ckpt.get("epoch", 0))
    print(f"✅ Resuming from epoch {last_epoch}; prev avg_gen_loss={ckpt.get('avg_gen_loss')} | prev avg_enc_loss={ckpt.get('avg_enc_loss')}")
    start_epoch = last_epoch + 1
else:
    print("▶️ Starting fresh inverse+summarizer training.")

# ---------- train loop (inverse + summarizer only) ----------
epochs_more = 3                  # how many NEW epochs to run from start_epoch
end_epoch   = start_epoch + epochs_more - 1
clip_norm   = 1.0
max_gen_len = 50

for ep in range(start_epoch, end_epoch + 1):
    personalized_model.train()
    pbar = tqdm(train_loader, desc=f"[INV+SUM] Epoch {ep}/{end_epoch}", dynamic_ncols=True)
    running_tot, running_enc, running_gen, seen = 0.0, 0.0, 0.0, 0

    for Bhist_batch, Blen_batch, Bpos_batch in pbar:
        optimizer.zero_grad(set_to_none=True)

        batch_tot, batch_enc, batch_gen, valid = 0.0, 0.0, 0.0, 0
        for i in range(len(Bhist_batch)):
            out = personalized_model(
                Bhist_batch[i], int(Blen_batch[i]), Bpos_batch[i],
                lookup_df, embed_tables, sid2sum, nid2body, tokenizer,
                max_len=max_gen_len, mode="train"
            )
            if out.get("skip", False):
                continue

            # Use gen loss for training; include enc loss only as a detached term
            gen_loss = out["gen_loss"]                                      # drives summarizer/inverse
            enc_loss = out["enc_loss"] if torch.is_tensor(out["enc_loss"]) else torch.tensor(float(out["enc_loss"]), device=device)
            loss = 0.4 * gen_loss + 0.6 * enc_loss.detach()                 # detach => no grads into encoder

            loss.backward()
            batch_tot += float(loss.detach().item())
            batch_enc += float(enc_loss.detach().item())
            batch_gen += float(gen_loss.detach().item())
            valid += 1

        if valid == 0:
            pbar.set_postfix_str("skip batch"); continue

        torch.nn.utils.clip_grad_norm_(trainable_params, clip_norm)
        optimizer.step()

        running_tot += batch_tot
        running_enc += batch_enc
        running_gen += batch_gen
        seen        += valid

        pbar.set_postfix({
            "enc": f"{running_enc / seen:.3f}",
            "gen": f"{running_gen / seen:.3f}",
            "tot": f"{running_tot / seen:.3f}",
        })

    # ---- save checkpoint ----
    ckpt_path = f"checkpoints/epoch_{ep}.pt"
    torch.save({
        "epoch": ep,
        "model_state": personalized_model.state_dict(),   # includes inverse + summarizer weights
        "optimizer_state": optimizer.state_dict(),
        "avg_total_loss": running_tot / max(1, seen),
        "avg_enc_loss":   running_enc / max(1, seen),
        "avg_gen_loss":   running_gen / max(1, seen),
    }, ckpt_path)
    print(f"💾 Saved inverse+summarizer checkpoint: {ckpt_path}")

    print("\nEpoch grad health summary (inverse+summarizer):")
    grad_health_report(personalized_model, limit_print=40)


✅ DataLoader: 38417 samples, batch_size=8
📦 Total params: 232,068,503
🟢 Trainable:   142,081,153
🧊 Frozen:      89,987,350


🔎 Found checkpoint: checkpoints/epoch_5.pt
⚠️ Skipping optimizer_state load: loaded state dict contains a parameter group that doesn't match the size of optimizer's group
✅ Resuming from epoch 5; prev avg_gen_loss=0.5976175675267787 | prev avg_enc_loss=2.9589636535152124


[INV+SUM] Epoch 6/8:   0%|          | 10/4803 [00:13<1:47:08,  1.34s/it, enc=3.088, gen=0.535, tot=2.067]


KeyboardInterrupt: 